# Quantify Tests

This notebook is an extension to the [number_of_flakes](number_of_flakes.ipynb) notebook. In this notebook, the key perfomance indicators that we would like to create greater visbility into and track over time is the percent of tests that passed/failed. This can help track and measure the effectiveness and success of our testing process.

By measuring this metric, we can observe the trend of test failures over time and a decrease in the percent of failures over time (or increase in percent of test passes), should correlate to improved test efficiencies, enhanced testing process and error free releases.  In this notebook, we derive the following metrics from the TestGrid dataset:

* total number of test cases
* number of test cases passed
* number of test cases failed
* percent of tests that pass
* percent of tests that fail

In [2]:
import gzip
import json
import os
import sys
import pandas as pd

sys.path.append("../../..")

module_path_1 = os.path.abspath(os.path.join("../../../data-sources/TestGrid"))
if module_path_1 not in sys.path:
    sys.path.append(module_path_1)

from ipynb.fs.defs.number_of_flakes import (  # noqa: E402
    testgrid_labelwise_encoding,
)  # noqa: E402

In [3]:
# Load test file
with gzip.open("../../../../data/raw/testgrid_183.json.gz", "rb") as read_file:
    testgrid_data = json.load(read_file)

In [4]:
failures_list = testgrid_labelwise_encoding(testgrid_data, 12)

In [5]:
len(failures_list)

27192485

In [6]:
failures_list[0]

(datetime.datetime(2021, 3, 15, 23, 40, 20),
 '"redhat-assisted-installer"',
 'periodic-ci-openshift-release-master-nightly-4.6-e2e-metal-assisted',
 'Overall',
 [{'metric': ['test-duration-minutes'],
   'values': [[80.28333333333333,
     92.05,
     80.98333333333333,
     117.71666666666667,
     108.63333333333334,
     87.06666666666666,
     111.03333333333333,
     100.28333333333333,
     143.2,
     577.0333333333333,
     18.833333333333332,
     92.81666666666666,
     81.36666666666666,
     89.05,
     84.08333333333333,
     108.68333333333334,
     82.61666666666666,
     99.98333333333333,
     99.63333333333334,
     84.35,
     80.15]]}],
 False)

In [7]:
# Convert to dataframe
failures_df = pd.DataFrame(
    failures_list, columns=["timestamp", "tab", "grid", "test", "graph", "failure"]
)
failures_df.head()

,timestamp,tab,grid,test,graph,failure
0,2021-03-15 23:40:20,"""redhat-assisted-installer""",periodic-ci-openshift-release-master-nightly-4...,Overall,"[{'metric': ['test-duration-minutes'], 'values...",False
1,2021-03-15 00:01:06,"""redhat-assisted-installer""",periodic-ci-openshift-release-master-nightly-4...,Overall,"[{'metric': ['test-duration-minutes'], 'values...",False
2,2021-03-13 20:51:32,"""redhat-assisted-installer""",periodic-ci-openshift-release-master-nightly-4...,Overall,"[{'metric': ['test-duration-minutes'], 'values...",False
3,2021-03-13 07:51:20,"""redhat-assisted-installer""",periodic-ci-openshift-release-master-nightly-4...,Overall,"[{'metric': ['test-duration-minutes'], 'values...",False
4,2021-03-13 06:43:20,"""redhat-assisted-installer""",periodic-ci-openshift-release-master-nightly-4...,Overall,"[{'metric': ['test-duration-minutes'], 'values...",False


In [8]:
# saving only the first 1000000 out of 19 million rows due to pvc limits.
# 250mb = 1 million --> 4750 mb = 19 million
failures_df.head(1000000).to_csv(
    "../../../../data/processed/failures.csv",
    header=False,
)

In [9]:
passing_list = testgrid_labelwise_encoding(testgrid_data, 1)

In [10]:
len(passing_list)

27192485

In [11]:
# Convert to dataframe
passing_df = pd.DataFrame(
    passing_list, columns=["timestamp", "tab", "grid", "test", "graph", "passing"]
)
passing_df.head()

,timestamp,tab,grid,test,graph,passing
0,2021-03-15 23:40:20,"""redhat-assisted-installer""",periodic-ci-openshift-release-master-nightly-4...,Overall,"[{'metric': ['test-duration-minutes'], 'values...",True
1,2021-03-15 00:01:06,"""redhat-assisted-installer""",periodic-ci-openshift-release-master-nightly-4...,Overall,"[{'metric': ['test-duration-minutes'], 'values...",True
2,2021-03-13 20:51:32,"""redhat-assisted-installer""",periodic-ci-openshift-release-master-nightly-4...,Overall,"[{'metric': ['test-duration-minutes'], 'values...",True
3,2021-03-13 07:51:20,"""redhat-assisted-installer""",periodic-ci-openshift-release-master-nightly-4...,Overall,"[{'metric': ['test-duration-minutes'], 'values...",True
4,2021-03-13 06:43:20,"""redhat-assisted-installer""",periodic-ci-openshift-release-master-nightly-4...,Overall,"[{'metric': ['test-duration-minutes'], 'values...",True


In [12]:
# saving only the first 1000000 out of 19 million rows due to pvc limits.
# 250mb = 1 million --> 4750 mb = 19 million
passing_df.head(1000000).to_csv(
    "../../../../data/processed/pass.csv",
    header=False,
)

In [13]:
# Metrics
no_tests = passing_df.head(1000000).test.count()
print("Total number of tests: %i" % (no_tests))
no_failures = failures_df.head(1000000).failure.sum()
print("Total number of failing tests: %i" % (no_failures))
test_failures_percentage = (
    (failures_df.head(1000000).failure.sum() / failures_df.head(1000000).test.count())
) * 100
print("Test failure percentage: %f" % (test_failures_percentage))
no_pass = passing_df.head(1000000).passing.sum()
print("Total number of passing tests: %i" % (no_pass))
test_pass_percentage = (
    (passing_df.head(1000000).passing.sum() / passing_df.head(1000000).passing.count())
) * 100
print("Test pass percentage: %f" % (test_pass_percentage))

Total number of tests: 1000000
Total number of failing tests: 4210
Test failure percentage: 0.421000
Total number of passing tests: 86976
Test pass percentage: 8.697600
